# Supervised Fine-Tuning

Import dependencies

In [1]:
from huggingface_hub import login as huggingface_hub_login
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, AutoConfig, AutoModelForSequenceClassification
import torch
from peft import LoraConfig, get_peft_model, TaskType
from dotenv import load_dotenv
import os
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score

/speed-scratch/a_s87063/repos/perf-pilot/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Log into Hugging Face

In [2]:
load_dotenv(dotenv_path="../secrets/.env")

hugging_face_token = os.getenv("HUGGING_FACE_TOKEN")

huggingface_hub_login(hugging_face_token)

Quantization config

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

Download the model

In [ ]:
!huggingface-cli login
!huggingface-cli download meta-llama/Meta-Llama-3-8B --local-dir ../LLMs/llama3-8b --local-dir-use-symlinks False

Load model and tokenizer

In [4]:
model_name = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config = AutoConfig.from_pretrained("meta-llama/Meta-Llama-3-8B", num_labels=2)

model = AutoModelForSequenceClassification.from_pretrained(
    "../LLMs/llama3-8b",
    config=config,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    ignore_mismatched_sizes=True
)

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.96s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ../LLMs/llama3-8b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Split the dataset into training and avaluation

In [5]:
dataset = load_dataset("json", data_files="../datasets/dataset.jsonl", split="train")

split_ratio = 0.8
split_index = int(len(dataset) * split_ratio)

train_dataset = dataset.select(range(0, split_index))
eval_dataset = dataset.select(range(split_index, len(dataset)))

# Shuffle ONLY the training set
train_dataset = train_dataset.shuffle(seed=42)

train_dataset.to_json("../datasets/train.jsonl", orient="records", lines=True)
eval_dataset.to_json("../datasets/eval.jsonl", orient="records", lines=True)

print(f"✅ Done! {len(train_dataset)} training / {len(eval_dataset)} evaluation samples.")


Creating json from Arrow format: 100%|██████████| 2/2 [00:00<00:00,  2.54ba/s]

✅ Done! 7444 training / 1862 evaluation samples.


Format and tokenize the dataset for classification

In [9]:
def format_for_classification(example):
    return {
        "text": example['prompt'],
        "label": int(example["response"])
    }

train_formatted_dataset = train_dataset.map(format_for_classification)
eval_formatted_dataset = train_dataset.map(format_for_classification)

def tokenize_class(example):
    encoding = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    encoding["labels"] = example["label"]
    return encoding

train_tokenized_dataset = train_formatted_dataset.map(tokenize_class, batched=True)
eval_tokenized_dataset = eval_formatted_dataset.map(tokenize_class, batched=True)

Map: 100%|██████████| 7444/7444 [00:01<00:00, 3794.51 examples/s]


ValueError: I/O operation on closed file

Apply LoRA with PEFT

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Define evaluation metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.exp(logits).sum(-1, keepdims=True)  # softmax
    preds = np.argmax(probs, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "auc": roc_auc_score(labels, probs[:, 1]),
    }

Set Up TrainingArguments and Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="./training/output/llama-classifier",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir="./training/logs",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=eval_tokenized_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Train the model

In [ ]:
trainer.train()

Save the fine-tuned model

In [ ]:
trainer.save_model("../LLMs/finetuned/classification/model/llama3-8B")
tokenizer.save_pretrained("../LLMs/finetuned/classification/tokenizer/llama3-8B")

Test the fine-tuned model

In [ ]:
predictions = trainer.predict(eval_tokenized_dataset)
logits = predictions.predictions
probs = np.exp(logits) / np.exp(logits).sum(-1, keepdims=True)

print("Probabilities for class 1:", probs[:, 1])
